In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from imp import reload
import warnings
from utils import util
from sklearn import model_selection
reload(util)
warnings.filterwarnings('ignore')

In [2]:
df_all = pd.read_csv('../data/20210809_encode.csv',index_col = 0)

In [3]:
df_all_stock = df_all[['stock_id']+df_all.columns[df_all.columns.str.endswith('_stock')].tolist()]
df_all_stock.drop_duplicates().to_pickle('../data/20210809.pkl')

In [4]:
params = {
    'n_estimators': 20000,
    'objective': 'rmse', # poisson
    'boosting_type': 'gbdt', # dart
    'max_depth': -1,
    'learning_rate': 0.01,
    'subsample': 0.8,
    'subsample_freq': 4,
    'feature_fraction': 0.8,
    'lambda_l1': 1,
    'lambda_l2': 1,
    'seed':46,
    'early_stopping_rounds': 500,
    'verbose': -1
} 

In [5]:
df_all.head(5)

time_id  Bnum  waproll_std5_B_mean  waproll_std5_B_std  \
0        5   159             0.000191            0.000104   
1       11   105             0.000150            0.000118   
2       16   130             0.000135            0.000061   
3       31   101             0.000139            0.000098   
4       62   104             0.000116            0.000076   

   waproll_std5_B_skew  waproll_std5_B_autocorr  waproll_std10_B_mean  \
0             0.984711                 0.748098              0.000244   
1             0.938263                 0.747746              0.000229   
2             0.716425                 0.730194              0.000157   
3             1.439628                 0.708243              0.000193   
4             0.922427                 0.788645              0.000158   

   waproll_std10_B_std  waproll_std10_B_skew  waproll_std10_B_autocorr  ...  \
0             0.000111              1.309386                  0.924732  ...   
1             0.000116              0.179212                  0.908148  ...   
2             0.000049              0.363781                  0.917034  ...   
3             0.000101              0.553793                  0.900866  ...   
4             0.000074              0.674093                  0.898002  ...   

   __book_wap2_lambda_____min___stock  \
0                            0.001142   
1                            0.001142   
2                            0.001142   
3                            0.001142   
4                            0.001142   

   __book_wap_mean_lambda_____mean___stock  \
0                                 0.004135   
1                                 0.004135   
2                                 0.004135   
3                                 0.004135   
4                                 0.004135   

   __book_wap_mean_lambda_____std___stock  \
0                                0.003384   
1                                0.003384   
2                                0.003384   
3                                0.003384   
4                                0.003384   

   __book_wap_mean_lambda_____max___stock  \
0                                 0.03336   
1                                 0.03336   
2                                 0.03336   
3                                 0.03336   
4                                 0.03336   

   __book_wap_mean_lambda_____min___stock  \
0                                0.000732   
1                                0.000732   
2                                0.000732   
3                                0.000732   
4                                0.000732   

   __book_wap_diff_lambda_____mean___stock  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   __book_wap_diff_lambda_____std___stock  \
0                                     NaN   
1                                     NaN   
2                                     NaN   
3                                     NaN   
4                                     NaN   

   __book_wap_diff_lambda_____max___stock  \
0                                     NaN   
1                                     NaN   
2                                     NaN   
3                                     NaN   
4                                     NaN   

   __book_wap_diff_lambda_____min___stock    target  
0                                 4.70394  0.004615  
1                                 4.70394  0.002474  
2                                 4.70394  0.002831  
3                                 4.70394  0.002201  
4                                 4.70394  0.002090  

[5 rows x 218 columns]

In [6]:
X_train = df_all[df_all.columns.difference(['target'])]
y_train = df_all['target']
features = df_all[df_all.columns.difference(['time_id','target'])].columns.tolist()
pd.DataFrame(features).to_pickle('../data/features_name_new.pkl')

In [7]:
oof_df = util.fit_model(params,X_train,y_train,features,cats=['stock_id'],n_fold=10,seed=66)

0it [00:00, ?it/s]

Training until validation scores don't improve for 500 rounds
[250]	training's rmse: 0.000506455	training's RMSPE: 0.234571	valid_1's rmse: 0.000513309	valid_1's RMSPE: 0.235949
[500]	training's rmse: 0.000476751	training's RMSPE: 0.220813	valid_1's rmse: 0.000485588	valid_1's RMSPE: 0.223207
[750]	training's rmse: 0.000465217	training's RMSPE: 0.21547	valid_1's rmse: 0.000476477	valid_1's RMSPE: 0.219019
[1000]	training's rmse: 0.000457122	training's RMSPE: 0.211721	valid_1's rmse: 0.00047089	valid_1's RMSPE: 0.21645
[1250]	training's rmse: 0.000450189	training's RMSPE: 0.20851	valid_1's rmse: 0.000466462	valid_1's RMSPE: 0.214415
[1500]	training's rmse: 0.000444549	training's RMSPE: 0.205898	valid_1's rmse: 0.000463038	valid_1's RMSPE: 0.212842
[1750]	training's rmse: 0.000439011	training's RMSPE: 0.203333	valid_1's rmse: 0.000459549	valid_1's RMSPE: 0.211238
[2000]	training's rmse: 0.000434786	training's RMSPE: 0.201376	valid_1's rmse: 0.000457572	valid_1's RMSPE: 0.210329
[2250]	tr

[17750]	training's rmse: 0.000337736	training's RMSPE: 0.156427	valid_1's rmse: 0.000435937	valid_1's RMSPE: 0.200384
[18000]	training's rmse: 0.000336826	training's RMSPE: 0.156005	valid_1's rmse: 0.000435966	valid_1's RMSPE: 0.200397
[18250]	training's rmse: 0.000335951	training's RMSPE: 0.155599	valid_1's rmse: 0.000435994	valid_1's RMSPE: 0.20041
[18500]	training's rmse: 0.000335046	training's RMSPE: 0.15518	valid_1's rmse: 0.000435899	valid_1's RMSPE: 0.200366
[18750]	training's rmse: 0.000334156	training's RMSPE: 0.154768	valid_1's rmse: 0.000435861	valid_1's RMSPE: 0.200349
[19000]	training's rmse: 0.000333237	training's RMSPE: 0.154343	valid_1's rmse: 0.000435801	valid_1's RMSPE: 0.200322
[19250]	training's rmse: 0.000332296	training's RMSPE: 0.153906	valid_1's rmse: 0.000435743	valid_1's RMSPE: 0.200295
[19500]	training's rmse: 0.00033141	training's RMSPE: 0.153496	valid_1's rmse: 0.000435661	valid_1's RMSPE: 0.200257
[19750]	training's rmse: 0.000330539	training's RMSPE: 0.15

1it [34:23, 2063.51s/it]

model saved 
Training until validation scores don't improve for 500 rounds
[250]	training's rmse: 0.000506499	training's RMSPE: 0.234475	valid_1's rmse: 0.00051103	valid_1's RMSPE: 0.235954
[500]	training's rmse: 0.000476779	training's RMSPE: 0.220717	valid_1's rmse: 0.000485548	valid_1's RMSPE: 0.224188
[750]	training's rmse: 0.000465832	training's RMSPE: 0.215649	valid_1's rmse: 0.000478014	valid_1's RMSPE: 0.22071
[1000]	training's rmse: 0.000457638	training's RMSPE: 0.211856	valid_1's rmse: 0.000472397	valid_1's RMSPE: 0.218116
[1250]	training's rmse: 0.000450808	training's RMSPE: 0.208694	valid_1's rmse: 0.000467912	valid_1's RMSPE: 0.216045
[1500]	training's rmse: 0.000444631	training's RMSPE: 0.205835	valid_1's rmse: 0.000463839	valid_1's RMSPE: 0.214165
[1750]	training's rmse: 0.000439566	training's RMSPE: 0.20349	valid_1's rmse: 0.000460891	valid_1's RMSPE: 0.212804
[2000]	training's rmse: 0.000435044	training's RMSPE: 0.201397	valid_1's rmse: 0.000458501	valid_1's RMSPE: 0.21

...

[17750]	training's rmse: 0.000337558	training's RMSPE: 0.156231	valid_1's rmse: 0.000436885	valid_1's RMSPE: 0.20214
[18000]	training's rmse: 0.000336649	training's RMSPE: 0.15581	valid_1's rmse: 0.000436795	valid_1's RMSPE: 0.202098
[18250]	training's rmse: 0.000335762	training's RMSPE: 0.1554	valid_1's rmse: 0.000436772	valid_1's RMSPE: 0.202088
[18500]	training's rmse: 0.000334867	training's RMSPE: 0.154985	valid_1's rmse: 0.000436704	valid_1's RMSPE: 0.202056
[18750]	training's rmse: 0.000333972	training's RMSPE: 0.154571	valid_1's rmse: 0.000436611	valid_1's RMSPE: 0.202013
[19000]	training's rmse: 0.000333068	training's RMSPE: 0.154153	valid_1's rmse: 0.000436579	valid_1's RMSPE: 0.201998
[19250]	training's rmse: 0.000332216	training's RMSPE: 0.153759	valid_1's rmse: 0.000436515	valid_1's RMSPE: 0.201969
[19500]	training's rmse: 0.000331316	training's RMSPE: 0.153342	valid_1's rmse: 0.000436442	valid_1's RMSPE: 0.201935
[19750]	training's rmse: 0.00033038	training's RMSPE: 0.1529

9it [3:39:50, 1441.55s/it]

model saved 
Training until validation scores don't improve for 500 rounds
[250]	training's rmse: 0.000507019	training's RMSPE: 0.234524	valid_1's rmse: 0.000530268	valid_1's RMSPE: 0.246639
[500]	training's rmse: 0.000477364	training's RMSPE: 0.220808	valid_1's rmse: 0.000503793	valid_1's RMSPE: 0.234325
[750]	training's rmse: 0.00046611	training's RMSPE: 0.215602	valid_1's rmse: 0.000495245	valid_1's RMSPE: 0.230349
[1000]	training's rmse: 0.00045784	training's RMSPE: 0.211777	valid_1's rmse: 0.000489402	valid_1's RMSPE: 0.227631
[1250]	training's rmse: 0.000451332	training's RMSPE: 0.208766	valid_1's rmse: 0.000485325	valid_1's RMSPE: 0.225735
[1500]	training's rmse: 0.000445382	training's RMSPE: 0.206014	valid_1's rmse: 0.000481365	valid_1's RMSPE: 0.223893
[1750]	training's rmse: 0.000440095	training's RMSPE: 0.203568	valid_1's rmse: 0.000477737	valid_1's RMSPE: 0.222206
[2000]	training's rmse: 0.00043582	training's RMSPE: 0.201591	valid_1's rmse: 0.000475504	valid_1's RMSPE: 0.22

10it [3:49:00, 1374.08s/it]

model saved 
0.20561931255122717
